In [1]:
from casadi import *

# Paper Cascaded Nonlinear MPC for Realtime Quadrotor Position Tracking. Schlagenhauf

# Define system, static/kinematic/dynamic model. Typically Differential (Algebraic) equations

# Expression graphs represent a computation in the computer memory. SX, MX


Ix = 0.0000166  #Moment of inertia around p_WB_W_x-axis, source: Julian Förster's ETH Bachelor Thesis
Iy = 0.0000167  #Moment of inertia around p_WB_W_y-axis, source: Julian Förster's ETH Bachelor Thesis
Iz = 0.00000293 #Moment of inertia around p_WB_W_z-axis, source: Julian Förster's ETH Bachelor Thesis
m = 0.029 #mass of Crazyflie 2.1
g = 9.81

Nx = 12
Nu = 4
Nhoriz = 10
T = 5. # time horizon

# x(t) = [x, y, z, phi, theta, psi, x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot]T
x = MX.sym("x", Nx)
u = MX.sym("u", Nu)


x_pos = x[0]  # x-position
y = x[1]  # y-position
z = x[2]  # z-position
phi = x[3]  # phi-angle, Euler angles
theta = x[4]  # theta-angle, Euler angles
psi = x[5]  # psi-angle, Euler angles
x_pos_dot = x[6]  # x velocity
y_dot = x[7]  # y velocity
z_dot = x[8]  # z velocity
phi_dot = x[9]  # phi_dot, angular velocity
theta_dot = x[10]  # theta_dot
psi_dot = x[11]  # psi-dot

thrust = u[0]
tau_phi = u[1]
tau_theta = u[2]
tau_psi = u[3]


# x˙(t) = [x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot, x_ddot, y_ddot, z_ddot, phi_ddot, ˙theta_ddot, psi_ddot]T


x_pos_ddot = (cos(phi) * sin(phi) * cos(psi) + sin(phi) * sin(phi))  * thrust / m
y_ddot = (cos(phi) * sin(phi) * cos(psi) - sin(phi) * sin(phi))  * thrust / m
z_ddot = -g + (cos(phi) * cos(theta)) * thrust / m
phi_ddot = theta_dot * psi_dot * (Iy - Iz) / (Ix) + tau_phi / Ix
theta_ddot = phi_dot * psi_dot * (Iz - Ix) / (Iy) + tau_phi / Iy
psi_ddot = theta_dot * phi_dot * (Ix - Iy) / (Iz) + tau_phi / Iz

x_dot = vertcat(x_pos_dot, y_dot, z_dot, phi_dot, theta_dot, psi_dot, x_pos_ddot, y_ddot, z_ddot, phi_ddot, theta_ddot, psi_ddot)

f = Function('f', [x, u], [x_dot], ['x', 'u'], ['x_dot'])

f

Function(f:(x[12],u[4])->(x_dot[12]) MXFunction)

In [ ]:
P = MX.sym('P',Nx + Nx) #parameters (which include the initial state and the reference state)

In [ ]:
obj = 0 # Objective function
g = []  # constraints vector

Q = diag(MX([100, 100, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
R = diag(MX([10.0, 10.0, 10.0, 10.0]))

st  = X(:,1); % initial state
g = [g;st-P(1:3)]; % initial condition constraints
for k = 1:N
    st = X(:,k);  con = U(:,k);
    obj = obj+(st-P(4:6))'*Q*(st-P(4:6)) + con'*R*con; % calculate obj
    st_next = X(:,k+1);
    k1 = f(st, con);   % new 
    k2 = f(st + h/2*k1, con); % new
    k3 = f(st + h/2*k2, con); % new
    k4 = f(st + h*k3, con); % new
    st_next_RK4=st +h/6*(k1 +2*k2 +2*k3 +k4); % new    

    g = [g;st_next-st_next_RK4]; # compute constraints % new
end

In [ ]:
# Objective term
J = 0

J = 0  # Initialize the cost function
g = [] # Initialize the list of constraints

# Loop over each control interval
for k in range(Nhoriz):
    # Control input at interval k
    Uk = U[:,k]
    
    # State at interval k
    Xk = X[:,k]
    
    # Integrate to the next interval
    Xk_next = rk4(f, Xk, Uk, DT)
    
    # Add to the objective function
    J += mtimes((Xk - x_r).T, Q, (Xk - x_r)) + mtimes((Uk - u_r).T, R, (Uk - u_r))
    
    # Add to the list of constraints
    g.append(Xk_next - X[:, k+1])
    
# Add terminal cost
J += mtimes((X[:,N] - x_r).T, Q_terminal, (X[:,N] - x_r))

In [2]:
# Fixed step Runge-Kutta 4 integrator
M = 4 # RK4 steps per interval
DT = T/Nhoriz/M
f = Function('f', [x, u], [xdot, J])
X0 = MX.sym('X0', 2)
U = MX.sym('U')
X = X0
Q = 0
for j in range(M):
   k1, k1_q = f(X, U)
   k2, k2_q = f(X + DT/2 * k1, U)
   k3, k3_q = f(X + DT/2 * k2, U)
   k4, k4_q = f(X + DT * k3, U)
   X=X+DT/6*(k1 +2*k2 +2*k3 +k4)
   Q = Q + DT/6*(k1_q + 2*k2_q + 2*k3_q + k4_q)
F = Function('F', [X0, U], [X, Q],['x0','p'],['xf','qf'])

In [15]:
# ---- objective          ---------


Q = diag(MX([100, 100, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
R = diag(MX([10.0, 10.0, 10.0, 10.0]))

# Add the objective function to the optimization problem
objective = 0  # Initialize the objective function

# Loop over the control horizon
for k in range(Nhoriz):
    # State tracking cost
    state_error = x[:, k] - Xref[:, k]  # State deviation
    objective += state_error.T @ Q @ state_error  # Quadratic cost for state deviation

    # Control effort cost
    if k < Nhoriz - 1:  # No control input for the last stage
        control_error = u[:, k]  # Assuming zero as the reference for control inputs
        objective += control_error.T @ R @ control_error  # Quadratic cost for control effort

# Terminal cost
terminal_error = x[:, Nhoriz] - Xref[:, Nhoriz]  # Terminal state deviation
objective += terminal_error.T @ Q @ terminal_error  # Quadratic cost for terminal state deviation

# Solve/deploy problem
# Numerical backends, 3rd-party solvers
# Ipopt

opti.minimize(objective)  # Set the objective in the optimization problem